In [10]:
import os
CURRENT_PATH = os.getcwd()

import pygame
import numpy as np

import sklearn
import torch
from torch.nn import functional as F
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from tqdm import tqdm

In [7]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.Conv = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)
        self.BN = torch.nn.BatchNorm2d(num_features=1)
        self.Pool = torch.nn.MaxPool2d(kernel_size=3, stride=1, padding=1)

        self.NN1 = torch.nn.Linear(1600, 400)
        self.NN2 = torch.nn.Linear(400, 128)
        self.NN3 = torch.nn.Linear(128, 64)
        self.NN4 = torch.nn.Linear(64, 2)


    def forward(self, input):
        x = self.Conv(input)
        x = self.BN(x)
        x = F.relu(x)
        x = self.Pool(x)
        x = F.dropout(x, p=0.2)
        x = torch.flatten(input, start_dim=1)

        x = self.NN1(x)
        x = F.relu(x)
        x = self.NN2(x)
        x = F.relu(x)
        x = self.NN3(x)
        x = F.relu(x)
        x = self.NN4(x)

        x[:, 0] = torch.tanh(x[:, 0])*150 + 50
        x[:, 1] = torch.tanh(x[:, 1])*30

        return x


model = CNN()
model.load_state_dict(torch.load(CURRENT_PATH + '/parameter/expertise.pt', map_location='cpu'))
model.eval()

CNN(
  (Conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (BN): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Pool): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
  (NN1): Linear(in_features=1600, out_features=400, bias=True)
  (NN2): Linear(in_features=400, out_features=128, bias=True)
  (NN3): Linear(in_features=128, out_features=64, bias=True)
  (NN4): Linear(in_features=64, out_features=2, bias=True)
)

In [ ]:
class PolicyNetContinuous(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(PolicyNetContinuous, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc_mu = torch.nn.Linear(hidden_dim, action_dim)
        self.fc_std = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        mu = 2.0 * torch.tanh(self.fc_mu(x))
        std = F.softplus(self.fc_std(x))
        return mu, std

class ValueNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim):
        super(ValueNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

class PPOContinuous:
    ''' 处理连续动作的PPO算法 '''
    def __init__(self, state_dim, hidden_dim, action_dim, actor_lr, critic_lr,
                 lmbda, epochs, eps, gamma, device):
        self.actor = PolicyNetContinuous(state_dim, hidden_dim,
                                         action_dim).to(device)
        self.critic = ValueNet(state_dim, hidden_dim).to(device)
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(),
                                                lr=actor_lr)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(),
                                                 lr=critic_lr)
        self.gamma = gamma
        self.lmbda = lmbda
        self.epochs = epochs
        self.eps = eps
        self.device = device

    def take_action(self, state):
        state = torch.tensor([state], dtype=torch.float).to(self.device)
        mu, sigma = self.actor(state)
        action_dist = torch.distributions.Normal(mu, sigma)
        action = action_dist.sample()
        return [action.item()]

    def update(self, transition_dict):
        states = torch.tensor(transition_dict['states'],
                              dtype=torch.float).to(self.device)
        actions = torch.tensor(transition_dict['actions'],
                               dtype=torch.float).view(-1, 1).to(self.device)
        rewards = torch.tensor(transition_dict['rewards'],
                               dtype=torch.float).view(-1, 1).to(self.device)
        next_states = torch.tensor(transition_dict['next_states'],
                                   dtype=torch.float).to(self.device)
        dones = torch.tensor(transition_dict['dones'],
                             dtype=torch.float).view(-1, 1).to(self.device)
        rewards = (rewards + 8.0) / 8.0  # 和TRPO一样,对奖励进行修改,方便训练
        td_target = rewards + self.gamma * self.critic(next_states) * (1 -
                                                                       dones)
        td_delta = td_target - self.critic(states)
        advantage = rl_utils.compute_advantage(self.gamma, self.lmbda,
                                               td_delta.cpu()).to(self.device)
        mu, std = self.actor(states)
        action_dists = torch.distributions.Normal(mu.detach(), std.detach())
        # 动作是正态分布
        old_log_probs = action_dists.log_prob(actions)

        for _ in range(self.epochs):
            mu, std = self.actor(states)
            action_dists = torch.distributions.Normal(mu, std)
            log_probs = action_dists.log_prob(actions)
            ratio = torch.exp(log_probs - old_log_probs)
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1 - self.eps, 1 + self.eps) * advantage
            actor_loss = torch.mean(-torch.min(surr1, surr2))
            critic_loss = torch.mean(
                F.mse_loss(self.critic(states), td_target.detach()))
            self.actor_optimizer.zero_grad()
            self.critic_optimizer.zero_grad()
            actor_loss.backward()
            critic_loss.backward()
            self.actor_optimizer.step()
            self.critic_optimizer.step()

In [ ]:
def train_off_policy_agent(env, new_agent, old_agent, num_episodes, replay_buffer, minimal_size, batch_size):
    #team_0 是训练的智能体，team_1 是陪练的智能体
    return_list = []
    for i in range(10):
        with tqdm(total=int(num_episodes/10), desc='Iteration %d' % i) as pbar:
            for i_episode in range(int(num_episodes/10)):
                episode_return = 0
                state = env.reset()
                done = False
                while not done:

                    action_team_0 = new_agent.take_action(state[0]['agent_obs'])
                    action_team_1 = old_agent.take_action(state[1]['agent_obs'])

                    next_state, reward, done, _ = env.step([action_team_0, action_team_1])
                    replay_buffer.add(state[0]['agent_obs'], action_team_0, reward, next_state, done)
                    state = next_state
                    episode_return += reward
                    if replay_buffer.size() > minimal_size:
                        b_s, b_a, b_r, b_ns, b_d = replay_buffer.sample(batch_size)
                        transition_dict = {'states': b_s, 'actions': b_a, 'next_states': b_ns, 'rewards': b_r, 'dones': b_d}
                        new_agent.update(transition_dict)
                return_list.append(episode_return)
                if (i_episode+1) % 10 == 0:
                    pbar.set_postfix({'episode': '%d' % (num_episodes/10 * i + i_episode+1), 'return': '%.3f' % np.mean(return_list[-10:])})
                pbar.update(1)
    return return_list

In [8]:
from olympics_engine.scenario import wrestling
from olympics_engine.generator import create_scenario
gamemap =  create_scenario('wrestling')        #load map config
game = wrestling(gamemap)

print('action dim 0 range = ', game.action_f)
print('action dim 1 range = ', game.action_theta)
our_team = 0        #we control team_0
obs = game.reset()
done = False
RENDER = False
if RENDER:
    game.render()

next_obs = obs
while not done:
    input = np.array([np.expand_dims(obs[0]['agent_obs'], axis=0)]).astype(np.float32)
    input = torch.tensor(input)
    output = model(input)
    #print(output)
    action_team_0 = output[0]    #for now we use random actions for both teams
    #action_team_0 = [np.random.uniform(-100,200), np.random.uniform(-30, 30)]
    action_team_1 = [np.random.uniform(-100,200), np.random.uniform(-30, 30)]
    
    if our_team:
        input_action = [action_team_1, action_team_0]
    else:
        input_action = [action_team_0, action_team_1]


    next_obs, reward, done, _ = game.step(input_action)

    if RENDER:
        game.render()
    
    obs = next_obs
print('final reward = ', reward)
pygame.display.quit()
pygame.quit()

action dim 0 range =  [-100, 200]
action dim 1 range =  [-30, 30]
final reward =  [1.0, 0]


In [ ]:
import pygame
import matplotlib.pyplot as plt

CURRENT_PATH = os.getcwd()

from olympics_engine.scenario import wrestling
from olympics_engine.generator import create_scenario
gamemap =  create_scenario('wrestling')        #load map config
game = wrestling(gamemap)

print('action dim 0 range = ', game.action_f)
print('action dim 1 range = ', game.action_theta)
our_team = 0        #we control team_0
obs = game.reset()
done = False
RENDER = False
if RENDER:
    game.render()

next_obs = obs

obs = np.array([np.expand_dims(obs[0]['agent_obs'], axis=0)]).astype(np.float32)
obs = torch.tensor(obs)
print(obs.shape)

out = model(obs)
print(out)